# DIC Crack Inspector

In [1]:
%matplotlib widget

In [2]:
from bmcs_shear.dic_crack import DICGrid, DICAlignedGrid, DICCOR, DICInspector, DICCrack

Convenience function to construct cracks

In [3]:
def construct_dic_cracks(cracks):
    '''Use the supplied list of crack specifications to construct the DICCrack objects'''
    dic_cracks = []
    for i, (ref_frame, rot_grid) in enumerate(cracks):
        dic_aligned_grid = DICAlignedGrid(dic_grid=dic_grid)
        dic_aligned_grid.trait_set(**ref_frame)
        dic_cor = DICCOR(dic_aligned_grid=dic_aligned_grid, crack_idx=i+1)
        dic_cor.trait_set(**rot_grid)
        dic_crack = DICCrack(name='crack #%d' % (i+1), dic_cor=dic_cor)
        dic_cracks.append(dic_crack)
    return dic_cracks

# B1_TV1

In [4]:
import numpy as np
phi = np.array([np.nan, 5, 6])
print(phi)
phi = np.where(np.isnan(phi), 0, phi)
phi

[nan  5.  6.]


array([0., 5., 6.])

In [5]:
B1_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -13, n_x_max = -10, n_x_step=1, n_y_min = 1, n_y_max=-1, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-11, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -20, n_x_max = -17, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-20, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min = 29, n_x_max = 37, n_x_step=1, n_y_min = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-28, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min=-35, n_x_max=-32, n_x_step=1, n_y_min = 2, n_y_max=12, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min=-46, n_x_max=-39, n_x_step=2, n_y_min = 6, n_y_max=-1, n_y_step = 2)),
    # crack 7
    (dict(y_ref_i=13, y_ref_j_min=1, y_ref_j_max=10),
    dict(n_x_min=1, n_x_max=8, n_x_step=2, n_y_min = 2, n_y_max=10, n_y_step = 2)),
]

In [6]:
dic_grid = DICGrid(n_x=59, n_y=15, d_x = 22, d_y = 22,  start_t = 0, end_t =31, U_factor = 1, 
                   dir_name='B1_TV1', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B1_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.0001233845375797716
evaluating step 2
phi_avg 0.00024277171988466414
evaluating step 3
phi_avg 0.00014929894375454876
evaluating step 4
phi_avg 0.0005327855174849936
evaluating step 5
phi_avg 0.00020150025575508984
evaluating step 6
phi_avg 0.000405893108300494
evaluating step 7
phi_avg 0.0004718082143479034
evaluating step 8
phi_avg 0.00035998427739640835
evaluating step 9
phi_avg 0.00026866499208620773
evaluating step 10
phi_avg 0.0003509554914606012
evaluating step 11
phi_avg 0.0004753689157832814
evaluating step 12
phi_avg 0.0006885045103288597
evaluating step 13
phi_avg 0.000796647569584839
evaluating step 14
phi_avg 0.0008336232620862558
evaluating step 15
phi_avg 0.0009107712501117837
evaluating step 16
phi_avg 0.0009401156041128896
evaluating step 17
phi_avg 0.0008868685811238237
evaluating step 18
phi_avg 0.0010200714285335357
evaluating step 19
phi_avg 0.0010319078088064254
evaluating step 20
phi_avg 0.0011516253908208

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00012712978085597875
evaluating step 2
phi_avg 0.00017772536766026922
evaluating step 3
phi_avg 0.0002894489686312618
evaluating step 4
phi_avg 0.0005207728317550671
evaluating step 5
phi_avg 0.0004564998337359963
evaluating step 6
phi_avg 0.0005475265878140894
evaluating step 7
phi_avg 0.0005491104426025734
evaluating step 8
phi_avg 0.0007367915033716983
evaluating step 9
phi_avg 0.0007672831273008871
evaluating step 10
phi_avg 0.0008013058910615073
evaluating step 11
phi_avg 0.0009103915158515804
evaluating step 12
phi_avg 0.0011541682140332609
evaluating step 13
phi_avg 0.0012561181889509908
evaluating step 14
phi_avg 0.0015480011080802253
evaluating step 15
phi_avg 0.0015560545844346219
evaluating step 16
phi_avg 0.0016031730327268694
evaluating step 17
phi_avg 0.0015989447503429028
evaluating step 18
phi_avg 0.0018181453649099121
evaluating step 19
phi_avg 0.0020941731984331887
evaluating step 20
phi_avg 0.002170235385000752
evaluating step 21
phi_avg 0.00222048614870698

In [7]:
dic_cor = dic_insp.dic_cracks.items[1].dic_cor

In [9]:
#dic_cor.dic_aligned_grid.X_ref_a

In [10]:
dic_cor.X_cor, dic_cor.X_cor_b

(array([114.1775775 , 211.06035055]), array([114.6137369 , 211.09543354]))

# B1_TV2

In [11]:
B1_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -12, n_x_step=1, n_y_min = 7, n_y_max=-1, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-23, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -31, n_x_max = -26, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-31, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = 20, n_x_max = 24, n_x_step=1, n_y_min = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=15, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=5, n_x_max=9, n_x_step=1, n_y_min = 7, n_y_max=12, n_y_step = 1)),
]

In [12]:
dic_grid = DICGrid(n_x=57, n_y=15, d_x = 22, d_y = 22,  start_t = 0, end_t =27, U_factor = 1, 
                   dir_name='B1_TV2', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B1_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0001440107082771716
evaluating step 2
phi_avg 0.00017614892450024095
evaluating step 3
phi_avg 0.00019347167112215944
evaluating step 4
phi_avg 0.0003413942876933685
evaluating step 5
phi_avg 0.0004260383070272038
evaluating step 6
phi_avg 0.0006842788296965581
evaluating step 7
phi_avg 0.0006416149611067749
evaluating step 8
phi_avg 0.0008100851757559435
evaluating step 9
phi_avg 0.0010559484958487964
evaluating step 10
phi_avg 0.0011832852209274815
evaluating step 11
phi_avg 0.0012408114763825069
evaluating step 12
phi_avg 0.001377117317343388
evaluating step 13
phi_avg 0.0014148363948246231
evaluating step 14
phi_avg 0.0016138793832686112
evaluating step 15
phi_avg 0.0016751261664892415
evaluating step 16
phi_avg 0.0015799212676221787
evaluating step 17
phi_avg 0.0015794508042288678
evaluating step 18
phi_avg 0.001705015847515498
evaluating step 19
phi_avg 0.0017749062610902483
evaluating step 20
phi_avg 0.0017990953481851345
evaluating step 21
phi_avg 0.001803160019762751

In [13]:
dic_grid.load_levels

array([  0.   ,  11.   ,  27.   ,  37.   ,  53.   ,  64.   ,  74.   ,
        84.   , 100.   , 116.   , 127.   , 138.   , 151.   , 161.5  ,
       175.   , 173.136, 170.93 , 173.6  , 185.351, 193.255, 195.103,
       195.381, 195.842, 196.132, 190.4  , 186.732, 185.528, 173.   ])

# B5_TV1

In [14]:
B5_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -12, n_x_max = -11, n_x_step=1, n_y_min = 1, n_y_max=-5, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-12, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -18, n_x_max = -16, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-18, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -23, n_x_max = -21, n_x_step=1, n_y_min = 5, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-23, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-30, n_x_max=-27, n_x_step=1, n_y_min = 1, n_y_max=8, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-30, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-37, n_x_max=-34, n_x_step=1, n_y_min = 1, n_y_max=8, n_y_step = 1)),
    # crack 7
    (dict(y_ref_i=-37, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-46, n_x_max=-42, n_x_step=1, n_y_min = 5, n_y_max=12, n_y_step = 1)),
    # crack 8
    (dict(y_ref_i=-44, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-52, n_x_max=-50, n_x_step=1, n_y_min = 5, n_y_max=12, n_y_step = 1)),
    # crack 9
    (dict(y_ref_i=-52, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-64, n_x_max=-58, n_x_step=1, n_y_min = 1, n_y_max=9, n_y_step = 1)),
]

In [15]:
dic_grid = DICGrid(n_x=68, n_y=11, d_x = 25, d_y = 25,  start_t = 0, end_t =23, U_factor = 1, 
                   dir_name='B5_TV1', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B5_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 9.372978864494115e-05
evaluating step 2
phi_avg 0.00013224782796186747
evaluating step 3
phi_avg 0.000143762780655268
evaluating step 4
phi_avg 8.996797533022274e-05
evaluating step 5
phi_avg 0.00018799461338346756
evaluating step 6
phi_avg 0.0002918692787976868
evaluating step 7
phi_avg 0.00018122639111186486
evaluating step 8
phi_avg 0.000264467043724477
evaluating step 9
phi_avg 0.0001555179882683589
evaluating step 10
phi_avg 0.000289954191017388
evaluating step 11
phi_avg 0.0003023367608899881
evaluating step 12
phi_avg 0.0003085191424442513
evaluating step 13
phi_avg 0.0004737219679996381
evaluating step 14
phi_avg 0.00043399204902870266
evaluating step 15
phi_avg 0.0006227232139911071
evaluating step 16
phi_avg 0.0006431309059314704
evaluating step 17
phi_avg 0.0008186933587950422
evaluating step 18
phi_avg 0.0009443020531816005
evaluating step 19
phi_avg 0.0010850392809255943
evaluating step 20
phi_avg 0.0011544109004046536
evaluating step 21
phi_avg 0.00126443387805202

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00011819153507312895
evaluating step 2
phi_avg 9.225472832336812e-05
evaluating step 3
phi_avg 0.0002602559260501087
evaluating step 4
phi_avg 0.00037005224800349665
evaluating step 5
phi_avg 0.0006209458414321778
evaluating step 6
phi_avg 0.0007384894447226566
evaluating step 7
phi_avg 0.0010156878841809948
evaluating step 8
phi_avg 0.00113225812162539
evaluating step 9
phi_avg 0.0013790771850809265
evaluating step 10
phi_avg 0.0014936550844846486
evaluating step 11
phi_avg 0.0017541028140886147
evaluating step 12
phi_avg 0.0019248477309075755
evaluating step 13
phi_avg 0.0021270068539444376
evaluating step 14
phi_avg 0.0022011218708608343
evaluating step 15
phi_avg 0.002393326415679588
evaluating step 16
phi_avg 0.0027023612126884245
evaluating step 17
phi_avg 0.0029548000448645093
evaluating step 18
phi_avg 0.003015685047307217
evaluating step 19
phi_avg 0.002953751646082587
evaluating step 20
phi_avg 0.0032626644885874107
evaluating step 21
phi_avg 0.003675817216433299
ev

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00018288425038785625
evaluating step 2
phi_avg 0.0001167237156138512
evaluating step 3
phi_avg 0.00017370645901043188
evaluating step 4
phi_avg 0.0001746870354425798
evaluating step 5
phi_avg 0.0002674387701140133
evaluating step 6
phi_avg 0.0004694104746953529
evaluating step 7
phi_avg 0.0004925976827904036
evaluating step 8
phi_avg 0.0005872202010014853
evaluating step 9
phi_avg 0.0006837536625482724
evaluating step 10
phi_avg 0.0007381405212318391
evaluating step 11
phi_avg 0.000845956030612487
evaluating step 12
phi_avg 0.0008679012156428296
evaluating step 13
phi_avg 0.000978518849589683
evaluating step 14
phi_avg 0.0010390709410305313
evaluating step 15
phi_avg 0.0011143871835916429
evaluating step 16
phi_avg 0.0013259458211239198
evaluating step 17
phi_avg 0.0013567674996204104
evaluating step 18
phi_avg 0.0014637884567345364
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00017703730360047617
evaluating step 2
phi_avg 0.00011557005977724559
evaluating step 3
phi_avg 0.00011024311801668869
evaluating step 4
phi_avg 0.00016293097242013523
evaluating step 5
phi_avg 0.00016368006571870726
evaluating step 6
phi_avg 0.00017516412339596833
evaluating step 7
phi_avg 0.0003312113313812752
evaluating step 8
phi_avg 0.0003839249943598165
evaluating step 9
phi_avg 0.0006072828443200231
evaluating step 10
phi_avg 0.0007045334860272026
evaluating step 11
phi_avg 0.000835419087286087
evaluating step 12
phi_avg 0.0009101978712143682
evaluating step 13
phi_avg 0.0009906719689710662
evaluating step 14
phi_avg 0.0011316177801537683
evaluating step 15
phi_avg 0.0012678664425755698
evaluating step 16
phi_avg 0.0014269591364718735
evaluating step 17
phi_avg 0.001571759238395346
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.000174246324798395
evaluating step 2
phi_avg 0.00012106381019825648
evaluating step 3
phi_avg 0.0002080977273652856
evaluating step 

# B5_TV2

In [16]:
B5_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -4, n_x_max = -3, n_x_step=1, n_y_min = 1, n_y_max=6, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-4, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -13, n_x_max = -11, n_x_step=1, n_y_min = 5, n_y_max=9, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-13, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -18, n_x_max = -16, n_x_step=1, n_y_min = 5, n_y_max=9, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-18, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -26, n_x_max = -23, n_x_step=1, n_y_min = 5, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-26, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-31, n_x_max=-28, n_x_step=1, n_y_min = 3, n_y_max=10, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-31, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-37, n_x_max=-35, n_x_step=1, n_y_min = 4, n_y_max=11, n_y_step = 1)),
    # crack 7
    (dict(y_ref_i=-37, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-46, n_x_max=-42, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 8
    (dict(y_ref_i=-46, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-51, n_x_max=-48, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 9
    (dict(y_ref_i=-52, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-64, n_x_max=-58, n_x_step=1, n_y_min = 2, n_y_max=9, n_y_step = 1)),
]

In [17]:
dic_grid = DICGrid(n_x=71, n_y=12, d_x = 25, d_y = 25,  start_t = 0, end_t =20, U_factor = 1, 
                   dir_name='B5_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B5_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00018566134924451828
evaluating step 2
phi_avg 0.00018468220326191994
evaluating step 3
phi_avg 0.00012654506483832217
evaluating step 4
phi_avg 0.00019890912893867288
evaluating step 5
phi_avg 0.0002607011128424897
evaluating step 6
phi_avg 0.0002296118631699311
evaluating step 7
phi_avg 0.00018072983198996902
evaluating step 8
phi_avg 0.00014031954862745295
evaluating step 9
phi_avg 0.0002951888635566861
evaluating step 10
phi_avg 0.00029438678698045476
evaluating step 11
phi_avg 0.0004498147716660763
evaluating step 12
phi_avg 0.0007180793077075285
evaluating step 13
phi_avg 0.0007814932824147926
evaluating step 14
phi_avg 0.0010128606819019986
evaluating step 15
phi_avg 0.0011737958523428986
evaluating step 16
phi_avg 0.0014411059958362913
evaluating step 17
phi_avg 0.0015304436357151334
evaluating step 18
phi_avg 0.0014235310887353283
evaluating step 19
phi_avg 0.0013610045693484402
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.0002756054371704956
evaluating 

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00010429148490289255
evaluating step 2
phi_avg 9.24447591415113e-05
evaluating step 3
phi_avg 0.00012224082322118883
evaluating step 4
phi_avg 0.00015315145870385558
evaluating step 5
phi_avg 0.00020508758648200154
evaluating step 6
phi_avg 0.00027349854051474885
evaluating step 7
phi_avg 0.0003484007997924713
evaluating step 8
phi_avg 0.0004697340771981196
evaluating step 9
phi_avg 0.0005417637903797548
evaluating step 10
phi_avg 0.0006204927380434688
evaluating step 11
phi_avg 0.0006830131250627304
evaluating step 12
phi_avg 0.0007407540555540507
evaluating step 13
phi_avg 0.0008507180634376866
evaluating step 14
phi_avg 0.0009339108133104259
evaluating step 15
phi_avg 0.0010334068674907884
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 8.749893589836179e-05
evaluating step 2
phi_avg 0.0002936531272306148
evaluating step 3
phi_avg 0.00015892928723625018
evaluating step 4
phi_avg 8.460655886940874e-05
evaluating step 5
phi_avg 0.0001280061940145674
evaluating step 6

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.00010213219030799884
evaluating step 2
phi_avg 0.00015288297465845422
evaluating step 3
phi_avg 0.00013625049094863827
evaluating step 4
phi_avg 0.00015909081367088522
evaluating step 5
phi_avg 8.200264081770733e-05
evaluating step 6
phi_avg 9.135694257464149e-05
evaluating step 7
phi_avg 0.00010632576999324261
evaluating step 8
phi_avg 0.00010316778133196152
evaluating step 9
phi_avg 0.00013533906466558265
evaluating step 10
phi_avg 0.00021665899518375514
evaluating step 11
phi_avg 0.00025188815838328557
evaluating step 12
phi_avg 0.0003211667840177329
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00019688363552935434
evaluating step 2
phi_avg 0.00010502418860424831
evaluating step 3
phi_avg 0.00014058218206631736
evaluating step 4
phi_avg 9.67861825585522e-05
evaluating step 5
phi_avg 0.00010372179047691352
evaluating step 6
phi_avg 0.00013358856503745644
evaluating step 7
phi_avg 0.0001346815726030172
evaluating step 8
phi_avg 0.00

# B6_TV1

In [19]:
B6_TV1_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -10, n_x_max = -9, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-10, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-15, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -20, n_x_max = -18, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-21, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -29, n_x_max = -23, n_x_step=1, n_y_min = 4, n_y_max=-1, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=31, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = 21, n_x_max = 26, n_x_step=1, n_y_min =3, n_y_max=-1, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=22, y_ref_j_min=1, y_ref_j_max=10),#22
     dict(n_x_min = 1, n_x_max = 14, n_x_step=2, n_y_min = 3, n_y_max=-3, n_y_step = 2))
]

In [20]:
dic_grid = DICGrid(n_x=58, n_y=15, d_x = 22, d_y = 22, start_t = 0, end_t =31, U_factor = 1, 
                   dir_name='B6_TV1', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B6_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.000293603696682505
evaluating step 2
phi_avg 0.00010205538070789475
evaluating step 3
phi_avg 0.0001350329329237713
evaluating step 4
phi_avg 0.0004638949639469766
evaluating step 5
phi_avg 0.0005251757571615416
evaluating step 6
phi_avg 0.0008102648500229816
evaluating step 7
phi_avg 0.0008380234389487917
evaluating step 8
phi_avg 0.0010289769630272877
evaluating step 9
phi_avg 0.001133919546355578
evaluating step 10
phi_avg 0.0014346684863106038
evaluating step 11
phi_avg 0.0016478485336659026
evaluating step 12
phi_avg 0.0018855375187524634
evaluating step 13
phi_avg 0.0020717249129138305
evaluating step 14
phi_avg 0.0023550771219620955
evaluating step 15
phi_avg 0.002314913212395358
evaluating step 16
phi_avg 0.002415000710798323
evaluating step 17
phi_avg 0.002489364001752873
evaluating step 18
phi_avg 0.0025682034774435736
evaluating step 19
phi_avg 0.0026305006734979133
evaluating step 20
phi_avg 0.002568000743278878
evaluating step 21
phi_avg 0.002537558919514966
eval

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00042682835542905926
evaluating step 2
phi_avg 0.00010770072500667944
evaluating step 3
phi_avg 0.0001994016444986651
evaluating step 4
phi_avg 0.00016536443482621124
evaluating step 5
phi_avg 0.00025894737791601986
evaluating step 6
phi_avg 0.00024603796021208525
evaluating step 7
phi_avg 0.00041677304110349724
evaluating step 8
phi_avg 0.00028106818876509485
evaluating step 9
phi_avg 0.0004928683122010816
evaluating step 10
phi_avg 0.0006049973006104474
evaluating step 11
phi_avg 0.0006024132752383322
evaluating step 12
phi_avg 0.0007608659089344575
evaluating step 13
phi_avg 0.0007627845501464372
evaluating step 14
phi_avg 0.0009277993511527509
evaluating step 15
phi_avg 0.0009307129423461435
evaluating step 16
phi_avg 0.0009270998381013785
evaluating step 17
phi_avg 0.0009066478739755655
evaluating step 18
phi_avg 0.0009648904475422854
evaluating step 19
phi_avg 0.0009975711172214583
evaluating step 20
phi_avg 0.0010109523720445502
evaluating step 21
phi_avg 0.00099981484

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00060087899910783
evaluating step 2
phi_avg 0.00032055507189109863
evaluating step 3
phi_avg 0.00019385605849259112
evaluating step 4
phi_avg 0.0002221920377230968
evaluating step 5
phi_avg 0.0002532690165198215
evaluating step 6
phi_avg 0.0002446984601827731
evaluating step 7
phi_avg 0.00023328036262314022
evaluating step 8
phi_avg 0.00036189475511624
evaluating step 9
phi_avg 0.00034006536057089624
evaluating step 10
phi_avg 0.0005893155098721006
evaluating step 11
phi_avg 0.0006191886676105118
evaluating step 12
phi_avg 0.0008277828571837314
evaluating step 13
phi_avg 0.0009410905120583
evaluating step 14
phi_avg 0.0010487681527359966
evaluating step 15
phi_avg 0.0009817244870564737
evaluating step 16
phi_avg 0.0010878071409666339
evaluating step 17
phi_avg 0.001208942279787139
evaluating step 18
phi_avg 0.0011403878621561053
evaluating step 19
phi_avg 0.0013252264102109718
evaluating step 20
phi_avg 0.0012179188219495157
evaluating step 21
phi_avg 0.0011659973894611476
ev

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00017957288314576238
evaluating step 2
phi_avg 0.00027168525649784035
evaluating step 3
phi_avg 0.0003968729455609492
evaluating step 4
phi_avg 0.00026691852022122736
evaluating step 5
phi_avg 0.00017873831557574233
evaluating step 6
phi_avg 0.00021829777534890517
evaluating step 7
phi_avg 0.00020582092950120024
evaluating step 8
phi_avg 0.0002260394133584773
evaluating step 9
phi_avg 0.000480972830763681
evaluating step 10
phi_avg 0.0003361651827022271
evaluating step 11
phi_avg 0.0003678937793613693
evaluating step 12
phi_avg 0.0004308751108299564
evaluating step 13
phi_avg 0.00043009201989993844
evaluating step 14
phi_avg 0.0005074842914474366
evaluating step 15
phi_avg 0.0005592289428543798
evaluating step 16
phi_avg 0.0005639996114102585
evaluating step 17
phi_avg 0.0006701329618840866
evaluating step 18
phi_avg 0.0004676346214581807
evaluating step 19
phi_avg 0.0005819452947074136
evaluating step 20
phi_avg 0.00045282968834384565
evaluating step 21
phi_avg 0.00059073946

# B6_TV2

In [21]:
B6_TV2_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -7, n_x_max = -4, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-7, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -12, n_x_step=1, n_y_min = 5, n_y_max=12, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-15, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -21, n_x_step=1, n_y_min = 7, n_y_max=12, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=20, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = 1, n_x_max = 14, n_x_step=1, n_y_min = 1, n_y_max=-1, n_y_step = 1)),
#     # crack 5
#     (dict(y_ref_i=31, y_ref_j_min=1, y_ref_j_max=10),
#      dict(n_x_min = 21, n_x_max = 26, n_x_step=1, n_y_min =3, n_y_max=-1, n_y_step = 1)),
#     # crack 6
#     (dict(y_ref_i=22, y_ref_j_min=1, y_ref_j_max=10),#22
#      dict(n_x_min = 1, n_x_max = 14, n_x_step=2, n_y_min = 3, n_y_max=-3, n_y_step = 2))
]

In [22]:
dic_grid = DICGrid(n_x=57, n_y=13, d_x = 22, d_y = 22, start_t = 0, end_t =19, U_factor = 1, 
                   dir_name='B6_TV2_', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B6_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00022506257880299877
evaluating step 2
phi_avg 0.00019406987150212435
evaluating step 3
phi_avg 0.00023340280220308303
evaluating step 4
phi_avg 0.0006220885121382383
evaluating step 5
phi_avg 0.0006568931269386032
evaluating step 6
phi_avg 0.0007508661167991786
evaluating step 7
phi_avg 0.0009055584734801897
evaluating step 8
phi_avg 0.0009831776807757212
evaluating step 9
phi_avg 0.0011839185653036858
evaluating step 10
phi_avg 0.0013477810883569872
evaluating step 11
phi_avg 0.0015024211568286665
evaluating step 12
phi_avg 0.001508353768110026
evaluating step 13
phi_avg 0.0014384571404709307
evaluating step 14
phi_avg 0.0016233350466001955
evaluating step 15
phi_avg 0.001623082168354761
evaluating step 16
phi_avg 0.0018065082322336213
evaluating step 17
phi_avg 0.001817846236552331
evaluating step 18
phi_avg 0.001789629632573321
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0004446237152185796
evaluating step 2
phi_avg 0.00026404712924430395
evaluating step 3
phi_avg 0.0004830781827446988
evaluating step 4
phi_avg 0.0003267543107849468
evaluating step 5
phi_avg 0.0004945641899579255
evaluating step 6
phi_avg 0.0006701777445511238
evaluating step 7
phi_avg 0.0007339529150137607
evaluating step 8
phi_avg 0.0007969129656012148
evaluating step 9
phi_avg 0.000915535539815621
evaluating step 10
phi_avg 0.0010980743885148807
evaluating step 11
phi_avg 0.001200574587910965
evaluating step 12
phi_avg 0.0013890865131119998
evaluating step 13
phi_avg 0.0013749318923206109
evaluating step 14
phi_avg 0.0015235098958791614
evaluating step 15
phi_avg 0.0016485511287301434
evaluating step 16
phi_avg 0.0015987916156641965
evaluating step 17
phi_avg 0.0018805527621696115
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00026638695519365146
evaluating step 2
phi_avg 0.00027131043578241507
evaluating step 3
phi_avg 0.00043488292540652315
evaluating step 4


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00026099604143910306
evaluating step 2
phi_avg 0.00015925674783084402
evaluating step 3
phi_avg 0.00019048333875222625
evaluating step 4
phi_avg 0.0002163998451915646
evaluating step 5
phi_avg 0.0002783508140823419
evaluating step 6
phi_avg 0.00036772209499444765
evaluating step 7
phi_avg 0.0003446880415190382
evaluating step 8
phi_avg 0.0003499436181449484
evaluating step 9
phi_avg 0.00032530498184364946
evaluating step 10
phi_avg 0.0003168333812305464
evaluating step 11
phi_avg 0.0003798911142021931
evaluating step 12
phi_avg 0.00048109403837722864
evaluating step 13
phi_avg 0.0005587267738936635
evaluating step 14
phi_avg 0.0004983539467083335
evaluating step 15
phi_avg 0.0006054375561208792


# B7_TV1

In [23]:
B7_TV1_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -12, n_x_max = -8, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-12, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -19, n_x_max = -16, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
     # crack 3
     (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 32, n_x_max = 35, n_x_step=1, n_y_min = 1, n_y_max=8, n_y_step = 1)),
     # crack 4
     (dict(y_ref_i=32, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 23, n_x_max = 26, n_x_step=1, n_y_min = 1, n_y_max=7, n_y_step = 1)),
     # crack 5
     (dict(y_ref_i=25, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 0, n_x_max = 18, n_x_step=2, n_y_min = 1, n_y_max=-1, n_y_step = 2)),
]

In [24]:
dic_grid = DICGrid(n_x=58, n_y=14, d_x = 22, d_y = 22, start_t = 0, end_t =25, U_factor = 1, 
                   dir_name='B7_TV1', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B7_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00011141290386465849
evaluating step 2
phi_avg 0.00013300995536972714
evaluating step 3
phi_avg 0.00031612524734117066
evaluating step 4
phi_avg 0.0006827830277979685
evaluating step 5
phi_avg 0.000840919012208924
evaluating step 6
phi_avg 0.001135403197348864
evaluating step 7
phi_avg 0.0013288220137698146
evaluating step 8
phi_avg 0.0017017972114850718
evaluating step 9
phi_avg 0.002027776671251026
evaluating step 10
phi_avg 0.0022778952881165257
evaluating step 11
phi_avg 0.0023261105625896716
evaluating step 12
phi_avg 0.002296081551005789
evaluating step 13
phi_avg 0.0024649415208334085
evaluating step 14
phi_avg 0.0026307182331956527
evaluating step 15
phi_avg 0.002510754590645288
evaluating step 16
phi_avg 0.0025554033734338883
evaluating step 17
phi_avg 0.0025427261774971788
evaluating step 18
phi_avg 0.00257567559180902
evaluating step 19
phi_avg 0.002586592076983337
evaluating step 20
phi_avg 0.002657674268942046
evalu

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.00011587510509033528
evaluating step 2
phi_avg 8.848986612788287e-05
evaluating step 3
phi_avg 0.00020686768670395207
evaluating step 4
phi_avg 0.0006280137790971821
evaluating step 5
phi_avg 0.0007080061968819317
evaluating step 6
phi_avg 0.0008065296294776015
evaluating step 7
phi_avg 0.000881785934560829
evaluating step 8
phi_avg 0.0011117447749438417
evaluating step 9
phi_avg 0.001342898979897236
evaluating step 10
phi_avg 0.0014645086384244427
evaluating step 11
phi_avg 0.0014883635500573089
evaluating step 12
phi_avg 0.001503472182119669
evaluating step 13
phi_avg 0.0014994796605752756
evaluating step 14
phi_avg 0.0015335313448746539
evaluating step 15
phi_avg 0.0016924517384684509
evaluating step 16
phi_avg 0.001574411596030668
evaluating step 17
phi_avg 0.0015194321960721872
evaluating step 18
phi_avg 0.0015867783923134951
evaluating step 19
phi_avg 0.0015838045148726945
evaluating step 20
phi_avg 0.001559316147902921
evaluating step 21
p

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.00015283495182353207
evaluating step 2
phi_avg 0.00018970818372580691
evaluating step 3
phi_avg 0.0002565710055545414
evaluating step 4
phi_avg 0.00028961596209030625
evaluating step 5
phi_avg 0.0002860997764840788
evaluating step 6
phi_avg 0.000514154139508621
evaluating step 7
phi_avg 0.0006617186838175688
evaluating step 8
phi_avg 0.0007434547513625881
evaluating step 9
phi_avg 0.0008234580078907862
evaluating step 10
phi_avg 0.0010175493491134473
evaluating step 11
phi_avg 0.0008838258550682933
evaluating step 12
phi_avg 0.0008881810192438172
evaluating step 13
phi_avg 0.000981152775808253
evaluating step 14
phi_avg 0.0009796520918866531
evaluating step 15
phi_avg 0.0011356289267335804
evaluating step 16
phi_avg 0.0010349164683505195
evaluating step 17
phi_avg 0.0009662432024815223
evaluating step 18
phi_avg 0.0009695381086518516
evaluating step 19
phi_avg 0.0009925032725041916
evaluating step 20
phi_avg 0.0009008124071562355
evaluating step 

# B7_TV2

In [25]:
B7_TV2_cracks = [
    # crask 1
    (dict(y_ref_i=-1, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -8, n_x_max = -6, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-8, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -13, n_x_max = -11, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
     # crack 3
     (dict(y_ref_i=-13, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = -23, n_x_max = -18, n_x_step=1, n_y_min = 3, n_y_max=8, n_y_step = 1)),
     # crack 4
     (dict(y_ref_i=-23, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = -28, n_x_max = -24, n_x_step=1, n_y_min = 3, n_y_max=7, n_y_step = 1)),
     # crack 5
     (dict(y_ref_i=-28, y_ref_j_min=1, y_ref_j_max=10),
      dict(n_x_min = 1, n_x_max = 12, n_x_step=1, n_y_min = 1, n_y_max=-1, n_y_step = 1)),
]

In [26]:
dic_grid = DICGrid(n_x=53, n_y=12, d_x = 23, d_y = 23, start_t = 0, end_t =15, U_factor = 1, 
                   dir_name='B7_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B7_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00018042559987524551
evaluating step 2
phi_avg 0.0002006973464519511
evaluating step 3
phi_avg 0.00011407478474649171
evaluating step 4
phi_avg 0.00010559218971922248
evaluating step 5
phi_avg 0.00036120955289678454
evaluating step 6
phi_avg 0.0005336206645646991
evaluating step 7
phi_avg 0.0007182781011176551
evaluating step 8
phi_avg 0.0008831663973533568
evaluating step 9
phi_avg 0.0010084803755225523
evaluating step 10
phi_avg 0.001025466726246144
evaluating step 11
phi_avg 0.001161754797790918
evaluating step 12
phi_avg 0.0013605320952935448
evaluating step 13
phi_avg 0.0014071067026064047
evaluating step 14
phi_avg 0.0015564390418429665
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00015040009373235724
evaluating step 2
phi_avg 9.169834643183831e-05
evaluating step 3
phi_avg 0.00020726281093839643
evaluating step 4
phi_avg 0.0005560691708006393
evaluating step 5
phi_avg 0.0006330785514163969
evaluating step 6
p

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0002608998922774604
evaluating step 2
phi_avg 0.0002453306749989491
evaluating step 3
phi_avg 0.00020944282867168325
evaluating step 4
phi_avg 0.00019458185413108908
evaluating step 5
phi_avg 0.0006037539985879925
evaluating step 6
phi_avg 0.0007268882011423872
evaluating step 7
phi_avg 0.0008743145084408679
evaluating step 8
phi_avg 0.0010232877811425102
evaluating step 9
phi_avg 0.0012381058499866093
evaluating step 10
phi_avg 0.0012061041693446795
evaluating step 11
phi_avg 0.0014070415351407353
evaluating step 12
phi_avg 0.0017126518669092628
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00015003173242170633
evaluating step 2
phi_avg 0.000510209867731805
evaluating step 3
phi_avg 0.00032149640548381726
evaluating step 4
phi_avg 0.0003950565419318289
evaluating step 5
phi_avg 0.0005058706852904002
evaluating step 6
phi_avg 0.0005165884609153363
evaluating step 7
phi_avg 0.000697567982766163
evaluating step 8
phi_avg 0.000799427670429075
evaluating step 9
phi_av

# B8_TV1

In [27]:
B8_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -8, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-13, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -19, n_x_max = -17, n_x_step=1, n_y_min = 8, n_y_max=-1, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -24, n_x_max = -21, n_x_step=1, n_y_min = 5, n_y_max=-1, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-24, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -33, n_x_max = -29, n_x_step=1, n_y_min = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=15, n_x_step=1, n_y_min = 2, n_y_max=12, n_y_step = 1)),
]

In [28]:
dic_grid = DICGrid(n_x=56, n_y=13, d_x = 23, d_y = 23,  start_t = 0, end_t =14, U_factor = 1, 
                   dir_name='B8_TV1', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B8_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 9.48314617354356e-05
evaluating step 2
phi_avg 0.00041579210623353847
evaluating step 3
phi_avg 0.00014594776082000512
evaluating step 4
phi_avg 0.0002830194702041944
evaluating step 5
phi_avg 0.0004094221929633157
evaluating step 6
phi_avg 0.0005983097942808978
evaluating step 7
phi_avg 0.000776678126959806
evaluating step 8
phi_avg 0.0009739994621828973
evaluating step 9
phi_avg 0.0011324248864598284
evaluating step 10
phi_avg 0.0013929463924097977
evaluating step 11
phi_avg 0.0015451241994845597
evaluating step 12
phi_avg 0.0018105465053917694
evaluating step 13
phi_avg 0.00192704209101195
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00024447246478967596
evaluating step 2
phi_avg 0.00016824094065373313
evaluating step 3
phi_avg 0.0002747286779520757
evaluating step 4
phi_avg 0.0003277452425221258
evaluating step 5
phi_avg 0.0004797462227602203
evaluating step 6
phi_avg 0.0005940739512724958
evaluating step 7
phi_av

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00018123393430352015
evaluating step 2
phi_avg 0.00017752694372637874
evaluating step 3
phi_avg 0.00010936429448178393
evaluating step 4
phi_avg 0.0001365537661055326
evaluating step 5
phi_avg 0.00013339678439360552
evaluating step 6
phi_avg 0.00019962759769374722
evaluating step 7
phi_avg 0.00022881926030111327
evaluating step 8
phi_avg 0.00016412797516856097
evaluating step 9
phi_avg 0.0002603865782997551
evaluating step 10
phi_avg 0.0002625819512891287
evaluating step 11
phi_avg 0.00035646752101581167
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 6.204549836295909e-05
evaluating step 2
phi_avg 0.00010189434710348483
evaluating step 3
phi_avg 0.0001905314982856403
evaluating step 4
phi_avg 0.00024254483955690395
evaluating step 5
phi_avg 0.0003068560962312404
evaluating step 6
phi_avg 0.0004457034221097528
evaluating step 7
phi_avg 0.0005579856682457753
evaluating step 8
phi_avg 0.0008003319141151967
evaluating step 9
phi_avg 0.0008617601664512043
evaluating step 

# B8_TV2

In [29]:
B8_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -8, n_x_max = -3, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-8, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 5, n_y_max=11, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-16, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -21, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-24, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -35, n_x_max = -31, n_x_step=1, n_y_min = 5, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 2, n_y_max=12, n_y_step = 1)),
]

In [30]:
dic_grid = DICGrid(n_x=56, n_y=14, d_x = 23, d_y = 23,  start_t = 0, end_t =15, U_factor = 1, 
                   dir_name='B8_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B8_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 9.060903285157222e-05
evaluating step 2
phi_avg 0.00022491418292242113
evaluating step 3
phi_avg 0.0004068793077469551
evaluating step 4
phi_avg 0.000626429754043621
evaluating step 5
phi_avg 0.0008256085398137159
evaluating step 6
phi_avg 0.001121485566110636
evaluating step 7
phi_avg 0.0013150163386231994
evaluating step 8
phi_avg 0.0016217254098150338
evaluating step 9
phi_avg 0.001811776800017737
evaluating step 10
phi_avg 0.002006305385211729
evaluating step 11
phi_avg 0.0022559860230635365
evaluating step 12
phi_avg 0.0024966186347272325
evaluating step 13
phi_avg 0.002702144792645716
evaluating step 14
phi_avg 0.0030301431040953425
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 8.772263721893118e-05
evaluating step 2
phi_avg 0.00011976955833883258
evaluating step 3
phi_avg 0.00024545684955257604
evaluating step 4
phi_avg 0.0004735380010241613
evaluating step 5
phi_avg 0.0006348235365345435
evaluating step 6
phi_avg 0.000831424943969695
evaluating step 7
phi_avg 

# B9_TV1

In [31]:
B9_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -9, n_x_max = -6, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-9, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -12, n_x_step=1, n_y_min = 1, n_y_max=4, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-14, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -19, n_x_max = -16, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -22, n_x_step=1, n_y_min = 3, n_y_max = 8, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-25, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-30, n_x_max=-28, n_x_step=1, n_y_min = 3, n_y_max=8, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-30, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
]

In [32]:
dic_grid = DICGrid(n_x=54, n_y=12, d_x = 23, d_y = 23, start_t = 0, end_t =18, U_factor = 1, 
                   dir_name='B9_TV1_', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B9_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00016216638311275421
evaluating step 2


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00014918939554631132
evaluating step 3
phi_avg 0.00031639436922208336
evaluating step 4
phi_avg 0.00030859886147312476
evaluating step 5
phi_avg 0.0003297485765047232
evaluating step 6
phi_avg 0.0005566507593045314
evaluating step 7
phi_avg 0.0006123749494793821
evaluating step 8
phi_avg 0.00078011716543488
evaluating step 9
phi_avg 0.0009763494410712457
evaluating step 10
phi_avg 0.0011090362486101324
evaluating step 11
phi_avg 0.0013138334192122289
evaluating step 12
phi_avg 0.0014088125307509176
evaluating step 13
phi_avg 0.0014122793489073668
evaluating step 14
phi_avg 0.0015800675864847747
evaluating step 15
phi_avg 0.0017522369840164752
evaluating step 16
phi_avg 0.0017492685628319605
evaluating step 17
phi_avg 0.0018089694157065574
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00029288935154208424
evaluating step 2
phi_avg 0.0003789926516589577
evaluating step 3
phi_avg 0.000294442231458687
evaluating step 4
phi_avg 0.0005048539162507564
evaluating step 5
phi_avg 0.0005823241442785564
evaluating step 6
phi_avg 0.001190147086453719
evaluating step 7
phi_avg 0.0036288031333709617
evaluating step 8
phi_avg 0.0019002885719604126
evaluating step 9
phi_avg 0.001789140602351612
evaluating step 10
phi_avg 0.0018619582557064425
evaluating step 11
phi_avg 0.0014511085558491811
evaluating step 12
phi_avg 0.0015117931057595825
evaluating step 13
phi_avg 0.0014081187776383436
evaluating step 14
phi_avg 0.0014974965051819857
evaluating step 15
phi_avg 0.0017284992656743216
evaluating step 16
phi_avg 0.0017535396902148435
evaluating step 0


...

phi_avg 0.0
evaluating step 1
phi_avg 0.00021074535870570793
evaluating step 2
phi_avg 0.00025780318564262364
evaluating step 3
phi_avg 0.0003772249556338764
evaluating step 4
phi_avg 0.00030292629542671393
evaluating step 5
phi_avg 0.00014194665738820298
evaluating step 6
phi_avg 0.0005701662947808064
evaluating step 7
phi_avg 0.0009335816027720739
evaluating step 8
phi_avg 0.0011315927210188202
evaluating step 9
phi_avg 0.0013699589492023087
evaluating step 10
phi_avg 0.0016063700079885855
evaluating step 11
phi_avg 0.0015082128818886124
evaluating step 12
phi_avg 0.0016509441146574306
evaluating step 13
phi_avg 0.001652657590783617
evaluating step 14
phi_avg 0.001897730908523382
evaluating step 15
phi_avg 0.002177722814039781
evaluating step 0


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.0003092486501733007
evaluating step 2
phi_avg 0.00024241610540695708
evaluating step 3
phi_avg 0.00024144546907140435
evaluating step 4
phi_avg 0.0008091169067992338
evaluating step 5
phi_avg 0.00028689628461073794
evaluating step 6
phi_avg 0.00046726130824211424
evaluating step 7
phi_avg 0.0003558689441226781
evaluating step 8
phi_avg 0.000473695696857216
evaluating step 9
phi_avg 0.0006093219791126345
evaluating step 10
phi_avg 0.0007936223870743294
evaluating step 11
phi_avg 0.001006897614980728
evaluating step 12
phi_avg 0.0010716882029514134
evaluating step 13
phi_avg 0.0010586477061242593
evaluating step 14
phi_avg 0.0010837843852314608
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.0007913423159449902
evaluating step 2
phi_avg 0.00024919181705295677
evaluating step 3
phi_avg 0.0003554510422271217
evaluating step 4
phi_avg 0.000647122256073366
evaluating step 5
phi_avg 0.00035597522024318043
evaluating step 6
phi_avg 0.0003678016

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00025417163612329903
evaluating step 2
phi_avg 0.00021733822341593573
evaluating step 3
phi_avg 0.00022534177559669393
evaluating step 4
phi_avg 0.0002061534332301122
evaluating step 5
phi_avg 0.0002398629203474157
evaluating step 6
phi_avg 0.0002300492842038118
evaluating step 7
phi_avg 0.0002466458182705637
evaluating step 8
phi_avg 0.0002489789816939101
evaluating step 9
phi_avg 0.000295835091511878
evaluating step 10
phi_avg 0.0004558395240008791
evaluating step 11
phi_avg 0.0006306968961522453
evaluating step 12
phi_avg 0.0010788277231452178


# B9_TV2

In [33]:
B9_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -9, n_x_max = -6, n_x_step=1, n_y_min = 5, n_y_max=-2, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-9, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -15, n_x_max = -11, n_x_step=1, n_y_min = 6, n_y_max=10, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-14, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -20, n_x_max = -17, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-19, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -22, n_x_step=1, n_y_min = 3, n_y_max = 8, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-25, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-33, n_x_max=-30, n_x_step=1, n_y_min = 6, n_y_max=10, n_y_step = 1)),
    # crack 6
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
]

In [34]:
dic_grid = DICGrid(n_x=57, n_y=13, d_x = 22, d_y = 22, start_t = 0, end_t =16, U_factor = 1, 
                   dir_name='B9_TV2', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B9_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 9.952042681228787e-05
evaluating step 2
phi_avg 0.00010548302991555993
evaluating step 3
phi_avg 0.0002654261577644264
evaluating step 4
phi_avg 0.00034318630228252994
evaluating step 5
phi_avg 0.000565376785457412
evaluating step 6
phi_avg 0.0007208055553099701
evaluating step 7
phi_avg 0.0009184350472513826
evaluating step 8
phi_avg 0.0013095010729629241
evaluating step 9
phi_avg 0.001439383795610855
evaluating step 10
phi_avg 0.001820283713683343
evaluating step 11
phi_avg 0.002068870818415064
evaluating step 12
phi_avg 0.0023520923048134437
evaluating step 13
phi_avg 0.0026096868019360574
evaluating step 14
phi_avg 0.0028707724947472564
evaluating step 15
phi_avg 0.0031053762037716167
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00020841245725877983
evaluating step 2
phi_avg 0.00017243854522010893
evaluating step 3
phi_avg 0.00020352616943178825
evaluating step 4
phi_avg 0.0002426894402646211
evaluating step 5
phi

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 9.251483020780558e-05
evaluating step 2
phi_avg 0.0002978006444295784
evaluating step 3
phi_avg 0.00022093350387546659
evaluating step 4
phi_avg 0.00019233983580544512
evaluating step 5
phi_avg 0.000299074708695254
evaluating step 6
phi_avg 0.0004332676192565511
evaluating step 7
phi_avg 0.0005530213506973315
evaluating step 8
phi_avg 0.0008009583149067248
evaluating step 9
phi_avg 0.0007740528736340321
evaluating step 10
phi_avg 0.001065253357756332
evaluating step 11
phi_avg 0.0009703571483328193
evaluating step 12
phi_avg 0.0010966448453647462
evaluating step 13
phi_avg 0.0012809965556911797
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00014199877700448924
evaluating step 2
phi_avg 0.00026778757681469704
evaluating step 3
phi_avg 0.00038123464051415304
evaluating step 4
phi_avg 0.00012055016820895743
evaluating step 5
phi_avg 0.00024973526353475826
evaluating step 6
phi_avg 0.00010787151295417905
evaluating step 7
phi_avg 0.0002319624406433748
evaluating step 8
phi_avg 0.00032816409317647726
evaluating step 9
phi_avg 0.0005172896038229374
evaluating step 10
phi_avg 0.0007821526484797297
evaluating step 11
phi_avg 0.000595526169338261
evaluating step 12
phi_avg 0.0007266151980956614
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00015840334069866405
evaluating step 2
phi_avg 0.00028902214145036515
evaluating step 3
phi_avg 0.000186335637565155
evaluating step 4
phi_avg 0.00012057336948512904
evaluating step 5
phi_avg 0.00020183817669434335
evaluating step 6
phi_avg 0.0005693765078767193
evaluating step 7
phi_avg 0.0003251409417868634
evaluating step 8
phi_avg 0.00039310757038449237
evaluating step

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00014273407002579337
evaluating step 2
phi_avg 0.0001569264978017126
evaluating step 3
phi_avg 0.00011461129465088612
evaluating step 4
phi_avg 0.00017796071716141653
evaluating step 5
phi_avg 0.00019476668825619273
evaluating step 6
phi_avg 0.00021236482943164252
evaluating step 7
phi_avg 0.0002410317810874941
evaluating step 8
phi_avg 0.00016059690455963013
evaluating step 9
phi_avg 0.0002467109848205844
evaluating step 10
phi_avg 0.00034544272239252764


# B10_TV1

In [35]:
B10_TV1_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -6, n_x_max = -3, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-6, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 6, n_y_max=10, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-14, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -25, n_x_max = -22, n_x_step=1, n_y_min = 5, n_y_max=10, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-25, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -33, n_x_max = -28, n_x_step=1, n_y_min = 7, n_y_max = 12, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-33, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=1, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=12, n_y_step = 1)),
]

In [36]:
dic_grid = DICGrid(n_x=57, n_y=14, d_x = 22, d_y = 22, start_t = 0, end_t =15, U_factor = 1, 
                   dir_name='B10_TV1', grid_column_first = True, grid_number_vertical = True)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B10_TV1_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00019145905435867693
evaluating step 2
phi_avg 0.001254696176576462
evaluating step 3
phi_avg 0.0024884723624373005
evaluating step 4
phi_avg 0.002313235260341996
evaluating step 5
phi_avg 0.0024865227965158235
evaluating step 6
phi_avg 0.0030493904474457084
evaluating step 7
phi_avg 0.004200067502043321
evaluating step 8
phi_avg 0.0025715661649576347
evaluating step 9
phi_avg 0.004187289318433293
evaluating step 10
phi_avg 0.0038942766947831727
evaluating step 11
phi_avg 0.003948274892105988
evaluating step 12
phi_avg 0.004135323244170747
evaluating step 13
phi_avg 0.0035030887150102833
evaluating step 14
phi_avg 0.0037720022479588258
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00013261549297445663
evaluating step 2
phi_avg 0.000176920330946592
evaluating step 3
phi_avg 0.0005252468068019927
evaluating step 4
phi_avg 0.0009964238482659796
evaluating step 5
phi_avg 0.0013340188092912841
evaluating step 6
phi_avg 0.

c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00028827825761778387
evaluating step 2
phi_avg 0.00018993593112479732
evaluating step 3
phi_avg 0.00035564192715344903
evaluating step 4
phi_avg 0.00040700203795196303
evaluating step 5
phi_avg 0.0005585599331875909
evaluating step 6
phi_avg 0.00074722612485147
evaluating step 7
phi_avg 0.0008442502787787062
evaluating step 8
phi_avg 0.001009668048618919
evaluating step 9
phi_avg 0.0009172093603642715
evaluating step 10
phi_avg 0.0010841406382118698
evaluating step 11
phi_avg 0.0011105990884470313
evaluating step 12
phi_avg 0.001358852221790102
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00021086525879563392
evaluating step 2
phi_avg 0.00041100276732816124
evaluating step 3
phi_avg 0.0002919294291647994
evaluating step 4
phi_avg 0.000353311447366805
evaluating step 5
phi_avg 0.0004383759120729785
evaluating step 6
phi_avg 0.0005660499323211656
evaluating step 7
phi_avg 0.0005550418412731924
evaluating step 8
phi_avg 0.0006307829223315434
evaluating step 9
phi_avg 0.0009064945062502224
evaluating step 10
phi_avg 0.0009347694239118565
evaluating step 11
phi_avg 0.0009915120798898744
evaluating step 0
phi_avg 0.0
evaluating step 1


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.00015780810383966905
evaluating step 2
phi_avg 0.00021112823534240422
evaluating step 3
phi_avg 0.0001477434065904804
evaluating step 4
phi_avg 0.0002395654379415439
evaluating step 5
phi_avg 0.0004222986787743298
evaluating step 6
phi_avg 0.0005702746297894646
evaluating step 7
phi_avg 0.0006894711264261504
evaluating step 8
phi_avg 0.0008503166065295955
evaluating step 9
phi_avg 0.0009811383002989425
evaluating step 10
phi_avg 0.0010515005426457598


# B10_TV2

In [37]:
B10_TV2_cracks = [
    # crack 1
    (dict(y_ref_i=-1 , y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -11, n_x_max = -8, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
    # crack 2
    (dict(y_ref_i=-11, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -16, n_x_max = -13, n_x_step=1, n_y_min = 1, n_y_max=4, n_y_step = 1)),
    # crack 3
    (dict(y_ref_i=-16, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -21, n_x_max = -18, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
    # crack 4
    (dict(y_ref_i=-21, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min = -29, n_x_max = -24, n_x_step=1, n_y_min = 1, n_y_max = 7, n_y_step = 1)),
    # crack 5
    (dict(y_ref_i=-29, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=-37, n_x_max=-33, n_x_step=1, n_y_min = 1, n_y_max=5, n_y_step = 1)),
     # crack 6
    (dict(y_ref_i=-37, y_ref_j_min=1, y_ref_j_max=10),
     dict(n_x_min=0, n_x_max=14, n_x_step=1, n_y_min = 1, n_y_max=10, n_y_step = 1)),
]

In [38]:
dic_grid = DICGrid(n_x=54, n_y=13, d_x = 23, d_y = 23, start_t = 0, end_t =10, U_factor = 1, 
                   dir_name='B10_TV2', grid_column_first = False, grid_number_vertical = False)
dic_insp = DICInspector(dic_grid=dic_grid)
dic_insp.dic_cracks.items = construct_dic_cracks(B10_TV2_cracks)
dic_insp.interact()

evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.00017545149186479097
evaluating step 2
phi_avg 0.0003913160348418573
evaluating step 3
phi_avg 0.0006368234345455437
evaluating step 4
phi_avg 0.0008729009125297093
evaluating step 5
phi_avg 0.001059374145230022
evaluating step 6
phi_avg 0.0013111483712865428
evaluating step 7
phi_avg 0.001638695402075936
evaluating step 8
phi_avg 0.0019059265938414533
evaluating step 9
phi_avg 0.001962270167191908
evaluating step 0


c:\users\fahad\pycharmprojects\bmcs_shear_zone\bmcs_shear\dic_crack\dic_cor.py:142: RuntimeWarning: invalid value encountered in true_divide
  phi = 2 * np.arctan(len_d_0t / len_d_0c)


phi_avg 0.0
evaluating step 1
phi_avg 0.0002191437146863288
evaluating step 2
phi_avg 0.000368141225260316
evaluating step 3
phi_avg 0.0004241326288931568
evaluating step 4
phi_avg 0.0004258725191167338
evaluating step 5
phi_avg 0.0007002040148946612
evaluating step 6
phi_avg 0.0006378608245061055
evaluating step 7
phi_avg 0.0007753762067577375
evaluating step 8
phi_avg 0.0008674212637481332
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.0003168467164471753
evaluating step 2
phi_avg 0.00039108681674453644
evaluating step 3
phi_avg 0.0002880506036932947
evaluating step 4
phi_avg 0.0004363420715320664
evaluating step 5
phi_avg 0.0006152172306577741
evaluating step 6
phi_avg 0.0007318803875277346
evaluating step 7
phi_avg 0.0008390322775951934
evaluating step 0
phi_avg 0.0
evaluating step 1
phi_avg 0.000125239960264578
evaluating step 2
phi_avg 0.0003331404076326799
evaluating step 3
phi_avg 0.0004191897694039965
evaluating step 4
phi_avg 0.0004248602542214293
evaluating step 5